In [1]:
!wget https://www.dropbox.com/s/uiurqzh02bo3dru/vine_labeled_cyberbullying_data.csv

--2019-08-06 09:30:58--  https://www.dropbox.com/s/uiurqzh02bo3dru/vine_labeled_cyberbullying_data.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/uiurqzh02bo3dru/vine_labeled_cyberbullying_data.csv [following]
--2019-08-06 09:30:59--  https://www.dropbox.com/s/raw/uiurqzh02bo3dru/vine_labeled_cyberbullying_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4bca977c6e6b3286b0f85f68c3.dl.dropboxusercontent.com/cd/0/inline/AmEQpBHh4B_T5qCzM0UwmaVHoa2oHJ_iB5CqwX2XR7G1nt8QJBuMSgMi-Q9PZ7QluQRiU9Jwg1uB3j4XBKhTgGiPQz6Gw92gq3pM0jo2GUvo7mrIcTg_uiwplD-aVR21EFE/file# [following]
--2019-08-06 09:30:59--  https://uc4bca977c6e6b3286b0f85f68c3.dl.dropboxusercontent.com/cd/0/inline/AmEQpBHh4B_T5qCzM0UwmaVHoa2oHJ_iB5CqwX2XR7

In [2]:
!wget https://www.dropbox.com/s/dbl1hdln6e4d5hv/comment_embedding.csv

--2019-08-06 09:31:04--  https://www.dropbox.com/s/dbl1hdln6e4d5hv/comment_embedding.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dbl1hdln6e4d5hv/comment_embedding.csv [following]
--2019-08-06 09:31:04--  https://www.dropbox.com/s/raw/dbl1hdln6e4d5hv/comment_embedding.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd90e17a45e34448e7dc23e4ab.dl.dropboxusercontent.com/cd/0/inline/AmF9fb4R-Mx12HyPUKdfgQwY9mQMJHuZxwiQ5BGiOUO2T_olXbf-AbkA04LXdHnCo2_DCRe3jlm0Xv66vA1ISi6X-TBuzqRlyby2jK2LuXJEwWBpYfEhAhSfbvUjjGDhiF0/file# [following]
--2019-08-06 09:31:05--  https://uccd90e17a45e34448e7dc23e4ab.dl.dropboxusercontent.com/cd/0/inline/AmF9fb4R-Mx12HyPUKdfgQwY9mQMJHuZxwiQ5BGiOUO2T_olXbf-AbkA04LXdHnCo2_DCRe3jlm0Xv66vA1I

In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import re

import tensorflow as tf
import keras
from keras.layers import Dense, LSTM, Bidirectional, Input, concatenate, Dropout
from keras.layers import BatchNormalization, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [0]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [0]:
data_shape = (10, 512)

In [0]:
class ModelDef:
    def __init__(self, input_shape=(99, 512), hidden_states=256):
        self.input_shape = input_shape
        self.hidden_states = hidden_states
    
    def build_resnet(self):
        inp = Input(shape=self.input_shape)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(inp)
        x1 = LeakyReLU(0.2)(x)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x1)
        x = LeakyReLU(0.2)(x)
        
        x = concatenate([x1, x])
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
        x1 = LeakyReLU(0.2)(x)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x1)
        x = LeakyReLU(0.2)(x)
        
        x = concatenate([x1, x])
        
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=False, kernel_initializer='glorot_uniform'))(x)
        x = LeakyReLU(0.2)(x)
        
        x = Dense(512, kernel_initializer='glorot_uniform')(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(x)
        
        model = Model(inputs=inp, outputs=x)
        return model
    
    def build_bilstm(self):
        inp = Input(shape=self.input_shape)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(inp)
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
        
        x1 = GlobalMaxPooling1D(x)
        x2 = GlobalAveragePooling1D(x)
        
        x = concatenate([x1, x2])
        
        x = Dense(512, kernel_initializer='glorot_uniform')(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(x)
        
        model = Model(inputs=inp, outputs=x)
        return model

In [16]:
model = ModelDef(data_shape)
model = model.build_resnet()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.summary()

W0806 09:32:20.187740 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 09:32:20.227793 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 09:32:20.236059 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 09:32:24.000184 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0806 09:32:24.026079 140368490915712 deprecation_wrappe

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 512)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 512)      1574912     input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 10, 512)      0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 10, 512)      1574912     leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [0]:
def convert_str_to_array(array_str):
    array_str = array_str.replace('[', '')
    array_str = array_str.replace(']', '')
    array_str = array_str.replace(' ', '')
    return np.fromstring(array_str, sep=', ')

class ReadData:
    def __init__(self, dataset, embedding_path, data_shape=(99, 512), train_val_split=0.1):
        print('Reading Dataset ..')
        self.dataset = pd.read_csv(dataset)
        self.dataset = self.dataset.sample(frac=1.0).reset_index(drop=True)
        print('Done.')
        print('Reading Embeddings ..')
        self.embeddings = pd.read_csv(embedding_path)
        self.embeddings.embedding = self.embeddings.embedding.apply(convert_str_to_array)
        print('Done.')
        
        self.training = self.dataset.head(int(len(self.dataset)*(1-train_val_split))).reset_index(drop=True)
        self.validation = self.dataset.tail(int(len(self.dataset)*(train_val_split))).reset_index(drop=True)
        
        self.train_size = len(self.training)
        self.val_size = len(self.validation)
        self.data_shape = data_shape
        
    def sent2vec(self, sent, embed_size=512):
        embed = self.embeddings[self.embeddings.comment == sent]
        if embed.empty:
            return np.zeros(embed_size)
        
        return list(embed.embedding)[0]
        
    def post_embedding(self, comments, max_len=99, dim=512):
        vector = []
        for i, comment in enumerate(comments):
            if i < max_len:
                try:
                    vec = self.sent2vec(comment, dim)
                    vector.append(vec)
                except Exception as e:
                    pass
            

        vectors = []
        vectors += list(vector)
        padding_len = max_len - len(vectors)
        for _ in range(padding_len):
            vectors.append(np.zeros(dim))

        return np.array(vectors)
    
    def get_comments(self, row, i):
        comments = []
        #print(row)
        for j in range(1, 100):
            comm = row['column{}'.format(j)][i]
            try:
                comm = [x.strip() for x in re.findall('\:\:.*?(.*)\(created', comm, re.MULTILINE | re.DOTALL)]
                #break
                if comm == []:
                    break
                else:
                    comments += comm
            except TypeError:
                pass
            
        return comments
    
    def generator_val(self, batch_size=64):
        while True:
            no_batches = int(self.val_size/batch_size)
            for i in range(no_batches):
                start_index = i*batch_size
                batch_x, batch_y = [], []
                for j in range(i, i+batch_size):
                    #print(j)
                    #print(self.training.head())
                    row = self.validation.iloc[[j]]
                    comments = self.get_comments(row, j)
                    x = self.post_embedding(comments, self.data_shape[0], self.data_shape[1])
                    y = []
                    if row['question2'][j] == 'noneBll':
                        y.append(0)
                    else:
                        y.append(1)

                    batch_x.append(x)
                    batch_y.append(y)

                x, y = np.array(batch_x), np.array(batch_y)
                #y = np.reshape(y, (y.shape[0], y.shape[2]))
                yield x, y

    def generator(self, batch_size=64):
        while True:
            no_batches = int(self.train_size/batch_size)
            for i in range(no_batches):
                start_index = i*batch_size
                batch_x, batch_y = [], []
                for j in range(i, i+batch_size):
                    #print(j)
                    #print(self.training.head())
                    row = self.training.iloc[[j]]
                    comments = self.get_comments(row, j)
                    x = self.post_embedding(comments, self.data_shape[0], self.data_shape[1])
                    y = []
                    if row['question2'][j] == 'noneBll':
                        y.append(0)
                    else:
                        y.append(1)

                    batch_x.append(x)
                    batch_y.append(y)

                x, y = np.array(batch_x), np.array(batch_y)
                #y = np.reshape(y, (y.shape[0], y.shape[2]))
                yield x, y
    
    def next_batch(self, i, batch_size=64):
        batch_x, batch_y = [], []
        for j in range(i, i+batch_size):
            #print(j)
            #print(self.training.head())
            row = self.training.iloc[[j]]
            comments = self.get_comments(row, j)
            x = self.post_embedding(comments, self.data_shape[0], self.data_shape[1])
            y = []
            if row['question2'][j] == 'noneBll':
                y.append(0)
            else:
                y.append(1)
                
            batch_x.append(x)
            batch_y.append(y)

        x, y = np.array(batch_x), np.array(batch_y)

        return x, y

In [0]:
batch_size=16
log_dir = 'post_classifier_{}-comments'.format(data_shape[0])
epochs = 20

In [11]:
dataset_path = 'vine_labeled_cyberbullying_data.csv'
reader = ReadData(dataset_path, 'comment_embedding.csv', data_shape=data_shape, train_val_split=0.2)

Reading Dataset ..
Done.
Reading Embeddings ..
Done.


In [0]:
train_generator = reader.generator(batch_size)
val_generator = reader.generator_val(batch_size)

In [0]:
# for x,y in train_generator:
#     print(x.shape, y.shape, x[0].shape)

In [14]:
logging = TrainValTensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(os.path.join(log_dir, 'ep{epoch:03d}-val_loss{val_loss:.3f}-val_acc{val_acc:.3f}.h5'),
        monitor='val_acc', save_weights_only=True, save_best_only=True, period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1)

print('Training on {} samples and Validating on {} samples.'.format(reader.train_size, reader.val_size))

Training on 776 samples and Validating on 194 samples.


In [0]:
model.fit_generator(generator=train_generator, steps_per_epoch=int(reader.train_size/batch_size),
                    validation_data=val_generator, epochs=epochs, validation_steps=int(reader.val_size/batch_size),
                    callbacks=[logging, checkpoint, reduce_lr, early_stopping])

W0806 09:32:39.646702 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0806 09:32:43.655260 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0806 09:32:43.656829 140368490915712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/20
48/48 [==============================] - 49s 1s/step - loss: 0.3383 - acc: 0.8880 - val_loss: 0.3310 - val_acc: 0.8698
Epoch 2/20
48/48 [==============================] - 42s 870ms/step - loss: 0.0668 - acc: 0.9844 - val_loss: 0.5229 - val_acc: 0.8490
Epoch 3/20
48/48 [==============================] - 43s 903ms/step - loss: 0.0228 - acc: 0.9948 - val_loss: 1.9721 - val_acc: 0.6406
Epoch 4/20
 6/48 [==>...........................] - ETA: 46s - loss: 0.2487 - acc: 0.9375

In [0]:
#!zip -r post_classifier.zip post_classifier

In [0]:
#!du -hs post_classifier/*

In [0]:
#!cp post_classifier/ep* .

In [0]:
'''# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1MNmLad9X0OQDa3H1nzjRKc5ckzxmEt_S'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentFile('comment_embedding.csv')))'''

In [0]:
#!wget https://www.dropbox.com/s/lxy5x0h7m7xcasn/vine_labeled_cyberbullying_data.csv